# 4. Data Modeling
* [4 Training Data](#2_Data_training_introduction)
  * [4.1 Dummy Variables/One Hot Encoding for Categorical](#3.1_one_hot_encoding)
  * [4.2 Standardize Numerical Data](#3.2_standardize)
  * [4.3 Testing Training](#3.3_testing_training)
 * [4.2 Summary](#3.7_Summary)

In [13]:
import sys
sys.executable

!/opt/homebrew/anaconda3/bin/python -m pip install xgboost fasttext nltk

In [7]:
%store -r products
%store -r X_train
%store -r X_test
%store -r y_train
%store -r y_test
#print(products) 
print(X_train.info())

<class 'pandas.core.frame.DataFrame'>
Index: 2661 entries, 9454 to 4138
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Brand           2661 non-null   category
 1   Description     2661 non-null   string  
 2   Keyword         2661 non-null   object  
 3   UPC             2661 non-null   object  
 4   MSRP            2661 non-null   float64 
 5   Quantity        2661 non-null   int64   
 6   SKU             2661 non-null   object  
 7   Color           2661 non-null   string  
 8   Size            2661 non-null   string  
 9   StyleNumber     2661 non-null   object  
 10  StyleName       2661 non-null   object  
 11  ParentCategory  2661 non-null   string  
dtypes: category(1), float64(1), int64(1), object(5), string(4)
memory usage: 259.3+ KB
None


Resources: 
- [blog post](https://towardsdatascience.com/document-embedding-techniques-fed3e7a6a25d)
- [textcategorizer](https://spacy.io/api/textcategorizer)
- [doc2vec](https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html#sphx-glr-auto-examples-tutorials-run-doc2vec-lee-py)
- [vec regression](https://towardsdatascience.com/multi-class-text-classification-with-doc2vec-logistic-regression-9da9947b43f4)

In [14]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix
import numpy as np
import gensim
from nltk.corpus import stopwords
from xgboost import XGBClassifier
import fasttext

# Load data
data = products

# Text preprocessing for 'Description'
stop_words = set(stopwords.words('english'))
vectorizer = CountVectorizer(stop_words=stop_words)
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words)
description_bow = vectorizer.fit_transform(data['Description'])
description_tfidf = tfidf_vectorizer.fit_transform(data['Description'])

# Encode categorical features
# encoder = OneHotEncoder(sparse_output=False)
# encoded_brand = encoder.fit_transform(data[['Brand']])
# encoded_category = encoder.fit_transform(data[['Category']])

# Combine features
combined_features_bow = np.hstack((description_bow.toarray(), encoded_brand, encoded_category))
combined_features_tfidf = np.hstack((description_tfidf.toarray(), encoded_brand, encoded_category))

# Split data into train and test sets
X_train_bow, X_test_bow, y_train, y_test = train_test_split(combined_features_bow, data['Category'], test_size=0.2, random_state=42)
X_train_tfidf, X_test_tfidf, _, _ = train_test_split(combined_features_tfidf, data['Category'], test_size=0.2, random_state=42)

# Define models
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Naive Bayes": MultinomialNB(),
    "Random Forest": RandomForestClassifier(),
    "Support Vector Machine": SVC(),
    "Doc2Vec": gensim.models.Doc2Vec(vector_size=100, min_count=2, epochs=40),
    "FastText": fasttext.train_supervised(input=description_tfidf, epoch=25, wordNgrams=2),
    "XGBoost": XGBClassifier()
}

# Evaluate each model
for name, model in models.items():
    if name == "Doc2Vec":
        # Train Doc2Vec model
        model.build_vocab(data['Description'])
        X_train_doc2vec = [model.infer_vector(doc.split()) for doc in data['Description'][:int(0.8 * len(data))]]
        X_test_doc2vec = [model.infer_vector(doc.split()) for doc in data['Description'][int(0.8 * len(data)):]]

        # Train the model
        model.fit(X_train_doc2vec, y_train)

        # Make predictions
        y_pred = model.predict(X_test_doc2vec)
    elif name == "FastText":
        # Train FastText model
        model.fit(X_train_tfidf, y_train)

        # Make predictions
        y_pred = model.predict(X_test_tfidf)
    else:
        # Train other models
        model.fit(X_train_bow, y_train)

        # Make predictions
        y_pred = model.predict(X_test_bow)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)

    # Calculate precision
    precision = precision_score(y_test, y_pred, average='weighted')

    # Print results
    print(f"Model: {name}")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix:\n{cm}\n")

InvalidParameterError: The 'stop_words' parameter of CountVectorizer must be a str among {'english'}, an instance of 'list' or None. Got {'s', 'wasn', 'here', 'a', "it's", 'no', 'it', 'of', 'myself', 'against', 'while', 'from', 'under', 'why', 'such', 'we', "shouldn't", 'doesn', 'our', 'that', 're', 'some', 'just', 'as', "hasn't", 'mustn', 'yourself', 'above', "hadn't", 'so', 'to', 'y', 'my', 've', 'most', 'other', 'ourselves', "mightn't", 'wouldn', "isn't", 'haven', 'which', 'mightn', 'ma', 'have', 'are', 'was', 'they', 'whom', 'didn', 'an', "you'll", 'i', 'do', 'very', "should've", "didn't", 'should', 'won', "that'll", 'did', 'and', 'down', 'same', 'aren', 'each', 'his', 'its', 'o', 'herself', 'hasn', 'until', "mustn't", 'out', 'll', 'shouldn', 'before', "couldn't", 'at', 'or', 'after', 'how', "wasn't", 'be', "weren't", 'where', 'doing', 'by', 'between', 'the', 'in', 't', 'me', 'both', 'into', 'who', 'because', 'too', 'can', 'what', 'weren', "won't", 'will', 'more', "she's", 'does', 